# Part 1: Packaging and Uploading your Algorithm for use with Amazon SageMaker

### An overview of Docker

If you're familiar with Docker already, you can skip ahead to the next section.

For many data scientists, Docker containers are a new concept, but they are not difficult, as you'll see here. 

Docker provides a simple way to package arbitrary code into an _image_ that is totally self-contained. Once you have an image, you can use Docker to run a _container_ based on that image. Running a container is just like running a program on the machine except that the container creates a fully self-contained environment for the program to run. Containers are isolated from each other and from the host environment, so the way you set up your program is the way it runs, no matter where you run it.

Docker is more powerful than environment managers like conda or virtualenv because (a) it is completely language independent and (b) it comprises your whole operating environment, including startup commands, environment variable, etc.

In some ways, a Docker container is like a virtual machine, but it is much lighter weight. For example, a program running in a container can start in less than a second and many containers can run on the same physical machine or virtual machine instance.

Docker uses a simple file called a `Dockerfile` to specify how the image is assembled. We'll see an example of that below. You can build your Docker images based on Docker images built by yourself or others, which can simplify things quite a bit.

Docker has become very popular in the programming and devops communities for its flexibility and well-defined specification of the code to be run. It is the underpinning of many services built in the past few years, such as [Amazon ECS].

Amazon SageMaker uses Docker to allow users to train and deploy arbitrary algorithms.

In Amazon SageMaker, Docker containers are invoked in a certain way for training and a slightly different way for hosting. The following sections outline how to build containers for the SageMaker environment.

Some helpful links:

* [Docker home page](http://www.docker.com)
* [Getting started with Docker](https://docs.docker.com/get-started/)
* [Dockerfile reference](https://docs.docker.com/engine/reference/builder/)
* [`docker run` reference](https://docs.docker.com/engine/reference/run/)

[Amazon ECS]: https://aws.amazon.com/ecs/

### How Amazon SageMaker runs your Docker container

Because you can run the same image in training or hosting, Amazon SageMaker runs your container with the argument `train` or `serve`. How your container processes this argument depends on the container:

* In the example here, we don't define an `ENTRYPOINT` in the Dockerfile so Docker will run the command `train` at training time and `serve` at serving time. In this example, we define these as executable Python scripts, but they could be any program that we want to start in that environment.
* If you specify a program as an `ENTRYPOINT` in the Dockerfile, that program will be run at startup and its first argument will be `train` or `serve`. The program can then look at that argument and decide what to do.
* If you are building separate containers for training and hosting (or building only for one or the other), you can define a program as an `ENTRYPOINT` in the Dockerfile and ignore (or verify) the first argument passed in. 

#### Running your container during training

When Amazon SageMaker runs training, your `train` script is run just like a regular Python program. A number of files are laid out for your use, under the `/opt/ml` directory:

    /opt/ml
    |-- input
    |   |-- config
    |   |   |-- hyperparameters.json
    |   |   `-- resourceConfig.json
    |   `-- data
    |       `-- <channel_name>
    |           `-- <input data>
    |-- model
    |   `-- <model files>
    `-- output
        `-- failure

##### The input

* `/opt/ml/input/config` contains information to control how your program runs. `hyperparameters.json` is a JSON-formatted dictionary of hyperparameter names to values. These values will always be strings, so you may need to convert them. `resourceConfig.json` is a JSON-formatted file that describes the network layout used for distributed training. Since scikit-learn doesn't support distributed training, we'll ignore it here.
* `/opt/ml/input/data/<channel_name>/` (for File mode) contains the input data for that channel. The channels are created based on the call to CreateTrainingJob but it's generally important that channels match what the algorithm expects. The files for each channel will be copied from S3 to this directory, preserving the tree structure indicated by the S3 key structure. 
* `/opt/ml/input/data/<channel_name>_<epoch_number>` (for Pipe mode) is the pipe for a given epoch. Epochs start at zero and go up by one each time you read them. There is no limit to the number of epochs that you can run, but you must close each pipe before reading the next epoch.

##### The output

* `/opt/ml/model/` is the directory where you write the model that your algorithm generates. Your model can be in any format that you want. It can be a single file or a whole directory tree. SageMaker will package any files in this directory into a compressed tar archive file. This file will be available at the S3 location returned in the `DescribeTrainingJob` result.
* `/opt/ml/output` is a directory where the algorithm can write a file `failure` that describes why the job failed. The contents of this file will be returned in the `FailureReason` field of the `DescribeTrainingJob` result. For jobs that succeed, there is no reason to write this file as it will be ignored.

#### Running your container during hosting

Hosting has a very different model than training because hosting is responding to inference requests that come in via HTTP. In this example, we use our recommended Python serving stack to provide robust and scalable serving of inference requests:

![Request serving stack](stack.png)

This stack is implemented in the sample code here and you can mostly just leave it alone. 

Amazon SageMaker uses two URLs in the container:

* `/ping` will receive `GET` requests from the infrastructure. Your program returns 200 if the container is up and accepting requests.
* `/invocations` is the endpoint that receives client inference `POST` requests. The format of the request and the response is up to the algorithm. If the client supplied `ContentType` and `Accept` headers, these will be passed in as well. 

The container will have the model files in the same place they were written during training:

    /opt/ml
    `-- model
        `-- <model files>



### The parts of the sample container

In the `container` directory are all the components you need to package the sample algorithm for Amazon SageMager:

    .
    |-- Dockerfile
    |-- build_and_push.sh
    `-- decision_trees
        |-- nginx.conf
        |-- predictor.py
        |-- serve
        |-- train
        `-- wsgi.py

Let's discuss each of these in turn:

* __`Dockerfile`__ describes how to build your Docker container image. More details below.
* __`build_and_push.sh`__ is a script that uses the Dockerfile to build your container images and then pushes it to ECR. We'll invoke the commands directly later in this notebook, but you can just copy and run the script for your own algorithms.
* __`decision_trees`__ is the directory which contains the files that will be installed in the container.
* __`local_test`__ is a directory that shows how to test your new container on any computer that can run Docker, including an Amazon SageMaker notebook instance. Using this method, you can quickly iterate using small datasets to eliminate any structural bugs before you use the container with Amazon SageMaker. We'll walk through local testing later in this notebook.

In this simple application, we only install five files in the container. You may only need that many or, if you have many supporting routines, you may wish to install more. These five show the standard structure of our Python containers, although you are free to choose a different toolset and therefore could have a different layout. If you're writing in a different programming language, you'll certainly have a different layout depending on the frameworks and tools you choose.

The files that we'll put in the container are:

* __`nginx.conf`__ is the configuration file for the nginx front-end. Generally, you should be able to take this file as-is.
* __`predictor.py`__ is the program that actually implements the Flask web server and the decision tree predictions for this app. You'll want to customize the actual prediction parts to your application. Since this algorithm is simple, we do all the processing here in this file, but you may choose to have separate files for implementing your custom logic.
* __`serve`__ is the program started when the container is started for hosting. It simply launches the gunicorn server which runs multiple instances of the Flask app defined in `predictor.py`. You should be able to take this file as-is.
* __`train`__ is the program that is invoked when the container is run for training. You will modify this program to implement your training algorithm.
* __`wsgi.py`__ is a small wrapper used to invoke the Flask app. You should be able to take this file as-is.

In summary, the two files you will probably want to change for your application are `train` and `predictor.py`.

### The Dockerfile

The Dockerfile describes the image that we want to build. You can think of it as describing the complete operating system installation of the system that you want to run. A Docker container running is quite a bit lighter than a full operating system, however, because it takes advantage of Linux on the host machine for the basic operations. 

For the Python science stack, we will start from a standard Ubuntu installation and run the normal tools to install the things needed by scikit-learn. Finally, we add the code that implements our specific algorithm to the container and set up the right environment to run under.

Along the way, we clean up extra space. This makes the container smaller and faster to start.

Let's look at the Dockerfile for the example:

## Code Cell 1

In [1]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 3 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM public.ecr.aws/bitnami/python:3.7

MAINTAINER Amazon AI <sage-learner@amazon.com>


RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python3-pip \
         python3-setuptools \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

RUN ln -s /usr/bin/python3 /usr/bin/python
RUN ln -sf /usr/bin/pip3 /usr/bin/pip

# Here we get all python packages.
# There's substantial overlap between scipy and numpy that we eliminate by
# linking them together. Likewise, pip leaves the install caches populated which uses
# a significant amount of space. These optimizations save a fair amount of space in the
# image, which reduces start up time.
RUN pip --no-cache-dir install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gunicorn

# Set so

### Building and registering the container

The following shell code shows how to build the container image using `docker build` and push the container image to ECR using `docker push`. This code is also available as the shell script `container/build-and-push.sh`, which you can run as `build-and-push.sh linear-regression_sample` to build the image `linear-regression_sample`. 

This code looks for an ECR repository in the account you're using and the current default region (if you're using a SageMaker notebook instance, this will be the region where the notebook instance was created). If the repository doesn't exist, the script will create it.

## Code Cell 2

In [2]:
# Install sm-docker and other necessary tools. 
# To install, simply use pip install within your notebook environment
!pip install setuptools
!pip install sagemaker-studio-image-build
!pip install matplotlib

  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'sagemaker-studio-image-build' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'sagemaker-studio-image-build'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for sagemaker-studio-image-build: filename=sagemaker_studio_image_build-0.6.0-py3-none-any.whl size=13558 sha256=06f6ad640ad75cde2992aff4457e02c5c9684e9da1d506012e125174a9c94f3f
  Stored in directory: /home/sagemaker-user/.cache/pip/wheels/b1/21/e2/17c1b6b9d114c0c0fe2e0e1c51a4bbfb3e96e5c2205396be8b
Successfully built sagemaker-studio-image-build


## Code Cell 3

In [3]:
%%sh


# The name of our algorithm
algorithm_name=sagemaker-linear-regression
cd container

chmod +x linear_regression/train
chmod +x linear_regression/serve

# The name of the image
fullname="${algorithm_name}:latest"

# Use in build sagemaker docker and codebuild to create image
sm-docker build . --role sagemaker_studio_role --repository ${fullname} --bucket lab-data-bucket-7f278e40

Created ECR repository sagemaker-linear-regression
....................[Container] 2025/08/31 16:47:32.449792 Running on CodeBuild On-demand

[Container] 2025/08/31 16:47:32.449802 Waiting for agent ping
[Container] 2025/08/31 16:47:34.658709 Waiting for DOWNLOAD_SOURCE
[Container] 2025/08/31 16:47:34.949683 Phase is DOWNLOAD_SOURCE
[Container] 2025/08/31 16:47:34.951077 CODEBUILD_SRC_DIR=/codebuild/output/src1629977189/src
[Container] 2025/08/31 16:47:34.951967 YAML location is /codebuild/output/src1629977189/src/buildspec.yml
[Container] 2025/08/31 16:47:34.956405 Setting HTTP client timeout to higher timeout for S3 source
[Container] 2025/08/31 16:47:34.956512 Processing environment variables
[Container] 2025/08/31 16:47:35.042755 No runtime version selected in buildspec.
[Container] 2025/08/31 16:47:35.060908 Moving to directory /codebuild/output/src1629977189/src
[Container] 2025/08/31 16:47:35.060929 Cache is not defined in the buildspec
[Container] 2025/08/31 16:47:35.104240 Ski

# Part 2: Review and prepare your data

Your data is loaded in the data/ folder. Review the data and split them into training and test sets.


## Code Cell 4

In [4]:
# Import Pandas and Numpy
# To learn more about Pandas, go to: https://pandas.pydata.org
# To learn more about Numpy, go to https://numpy.org/
import os
import pandas as pd
import numpy as np

This dataset has five cloumns. This datasets were created for the demostration purpose only.

* `book_id` - Unique id number of the book
* `customer_ratings` - In the scale of 1-5, received customer ratings.
* `helpful_votes` - The ratings that were postive to the product.
* `total_votes` - Total number of ratings
* `price` - Listed price

## Code Cell 5

In [5]:
# Read the file book_data.csv as a dataFrame
df = pd.read_csv('data/book_data.csv')

# Print the first 5 lines of the dataFrame.
df.head()

,book_id,customer_ratings,helpful_votes,total_votes,price
0,48541186,5,16,20,2.638007
1,52253037,5,1,1,2.662567
2,52534781,4,16,19,2.531479
3,25947084,5,2,2,2.572494
4,37527885,5,1,1,2.662567


An important part of training a machine learning model is splitting the data into training, validation, and test subsets. You will use the `train_test_split()` function from the `sklearn` library ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)).

## Code Cell 6

In [6]:
# Import train_test_split module from Scikit-learn 
# To learn more about data split, 
# go to https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

from sklearn.model_selection import train_test_split
train, test = train_test_split(df,test_size=0.2)

print(f"Length of train_data is: {train.shape}")
print(f"Length of test_data is: {test.shape}")

Length of train_data is: (3999, 5)
Length of test_data is: (1000, 5)


## Code Cell 7

In [7]:
# Converting the array to the data frame
train_df = pd.DataFrame(train)
test_df = pd.DataFrame(test)

# Save the train_df frames as train_data.csv
train_df.to_csv('data/train_data.csv', header=False, index = False)


# Save the test_df frames as test_data.csv
test_df.to_csv('data/test_data.csv', header=False, index = False)

## Code Cell 8

In [8]:
%%sh

# Move the train and test data to data/train and data/test folder respectively

cd data/
mkdir test
mkdir train
mv train_data.csv train/
mv test_data.csv test/

# Part 3: Using your Algorithm in Amazon SageMaker

Once you have your container packaged and data reviewed, you can use it to train models and use the model for hosting .

## Set up the environment

Here we specify a bucket to use and the role that will be used for working with SageMaker.

## Code Cell 9

In [9]:
# S3 bucket name and prefix
bucket_name = "lab-data-bucket-7f278e40"
prefix = "DEMO-scikit-book-price"

# Import boto3 and define IAM role
import boto3
from sagemaker import get_execution_role

role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Create the session

The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

## Code Cell 10

In [10]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Upload the data for training

When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we're using fictitious book ratings data, which we have included. 

We can use use the tools provided by the SageMaker Python SDK to upload the data to a default bucket. 

## Code Cell 11

In [11]:
WORK_DIRECTORY = "data/train"

data_location = sess.upload_data(WORK_DIRECTORY, bucket=bucket_name, key_prefix=prefix)

## Create an estimator and fit the model

In order to use SageMaker to fit our algorithm, we'll create an `Estimator` that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

* The __container name__. This is constructed as in the shell commands above.
* The __role__. As defined above.
* The __instance count__ which is the number of machines to use for training.
* The __instance type__ which is the type of machine to use for training.
* The __output path__ determines where the model artifact will be written.
* The __session__ is the SageMaker session object that we defined above.

Then we use fit() on the estimator to train against the data that we uploaded above.

## Code Cell 12

In [12]:
output_path=f"s3://{bucket_name}/output"
print(output_path)

s3://lab-data-bucket-7f278e40/output


## Code Cell 13

In [13]:
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name
image = f"{account}.dkr.ecr.{region}.amazonaws.com/sagemaker-linear-regression:latest"
print(image)

linear_regression = sage.estimator.Estimator(
    image,
    role,
    1,
    "ml.m5.large",
    output_path=f"s3://{bucket_name}/output",
    sagemaker_session=sess,
)

linear_regression.fit(data_location)

INFO:sagemaker:Creating training-job with name: sagemaker-linear-regression-2025-08-31-16-54-29-311


140204528665.dkr.ecr.us-east-1.amazonaws.com/sagemaker-linear-regression:latest
2025-08-31 16:54:30 Starting - Starting the training job...
2025-08-31 16:54:44 Starting - Preparing the instances for training...
2025-08-31 16:55:31 Downloading - Downloading the training image..Starting the training.
Training complete.

2025-08-31 16:56:00 Training - Training image download completed. Training in progress.
2025-08-31 16:56:00 Uploading - Uploading generated training model
2025-08-31 16:56:00 Completed - Training job completed
Training seconds: 54
Billable seconds: 54


## Hosting your model
You can use a trained model to get real time predictions using HTTP endpoint. Follow these steps to walk you through the process.

### Deploy the model

Deploying the model to SageMaker hosting just requires a `deploy` call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

## Code Cell 14

In [15]:
from sagemaker.serializers import CSVSerializer
predictor = linear_regression.deploy(1, "ml.m5.large", serializer=CSVSerializer())

INFO:sagemaker:Creating model with name: sagemaker-linear-regression-2025-08-31-16-59-41-524
INFO:sagemaker:Creating endpoint-config with name sagemaker-linear-regression-2025-08-31-16-59-41-524
INFO:sagemaker:Creating endpoint with name sagemaker-linear-regression-2025-08-31-16-59-41-524


----!

# Part 4: Validate the Endpoint

### Use test data for a prediction

In order to do some predictions, we'll use test data and do predictions against it. This is a way to see how the mechanism works.

## Code Cell 15

In [16]:
test_data = pd.read_csv("data/test/test_data.csv")

## Code Cell 16

In [17]:
x_data = test_data.iloc[:,1:4]
y_data = test_data.iloc[:,-1]

Prediction is as easy as calling predict with the predictor we got back from deploy and the data we want to do predictions with. The serializers take care of doing the data conversions for us.

## Code Cell 17

In [18]:
predictions = predictor.predict(x_data.values).decode("utf-8")

## Code Cell 18

In [19]:
# Converting the recived prediction results into a list.

def Convert(string):
    l = list(string.split("\n"))
    return l

predictions = Convert(predictions)

# Removing the last empty value
predictions.pop()

# Print the predicton value as a list
print(predictions)

['1.851258349', '2.874678916', '2.560099599000004', '2.6344688610000007', '2.551895470999998', '2.731588765', '2.5999741699999985', '3.000429739', '2.5724935929999955', '2.6716778709999907', '2.460634476', '2.424881637', '2.63748973', '2.11058971', '2.5724935929999955', '2.5724935929999955', '2.531223905999998', '2.560099599000004', '2.7428607130000002', '2.6716778709999907', '2.442479769', '2.5973031480000004', '2.11058971', '2.5973031480000004', '2.6625674210000043', '2.6625674210000043', '2.4629696230000016', '3.0398105539999998', '2.588739392', '2.094982732', '2.558950494999999', '2.561271762', '2.6625674210000043', '2.809926683', '2.747780686', '2.531223905999998', '2.6716778709999907', '2.560099599000004', '2.63748973', '2.432969291', '2.6716778709999907', '2.5724935929999955', '2.6716778709999907', '3.003029471', '2.5484804269999963', '2.460634476', '2.560099599000004', '2.511241467', '2.7235228869999997', '2.751185751000001', '2.6127805150000003', '2.625210001', '2.671677870999

#### Review the predicted result and the test result in a scatter plot

Scatter plot is a graph in which the values of two variables are plotted along two axes, the pattern of the resulting points revealing any correlation present.

## Code Cell 19

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

## Code Cell 20

In [ ]:
# Convert the y_data array to a list
y_list = y_data.tolist()

## Code Cell 21

In [ ]:
plt.scatter(predictions, y_list)

### Optional cleanup
When you're done with the endpoint, you'll want to clean it up.

## Code Cell 22

In [ ]:
# predictor.delete_model()
# predictor.delete_endpoint()